In [87]:
import string
import math
from collections import OrderedDict
import numpy as np
from operator import itemgetter

In [88]:
def multVet(d,q):
    soma = 0
    for i in range(0,len(q)):
        soma += d[i]*q[i]
    return soma

def norma(v):
    soma = 0
    for i in range(0,len(v)):
        soma += v[i]**2
    return math.sqrt(soma)

def sumStr(s):
    return sum(map(int, s))

def getRelevantes(qn):
    relevantes = perguntas[qn][perguntas[qn].index('RD')+2:].split()
    docsRlv = []
    for i in range(0,len(relevantes)-1,2):
        docsRlv.append([relevantes[i],sumStr(relevantes[i+1])])
    return docsRlv

# adiciona as palavras no dicionário: voc

In [89]:
cfcs = ['cf74','cf75','cf76','cf77','cf78','cf79']
listaInv = {}
docs = []
for i in range(0,6):
    
    arq = open('cfc/' + cfcs[i])
    documentos = arq.read()
    documentos = documentos.split('\n\n')
    
    for documento in documentos:
        voc = {}
        if "" in documento:
            break
        if len(documento) < 5:
            #print('erro')
            continue
        else:
            try:
                autor = documento[documento.index('\nAU')+3:documento.index('\nTI')].split()
            except:
                autor = []
                
            try:
                title = documento[documento.index('\nTI')+3:documento.index('\nSO')].split()
            except:
                title = []
                
#             try:
#                 fonte = documento[documento.index('\nSO')+3:documento.index('\nMJ')].split()
#             except:
#                 fonte = []
                
            try:
                mj = documento[documento.index('\nMJ')+3:documento.index('\nMN')].split()
            except:
                mj = []
                
            try:
                mn = documento[documento.index('MN')+3:documento.index('\nAB')].split()
            except:
                mn = documento[documento.index('MN')+3:documento.index('\nEX')].split() 
                
            try:
                try:
                    ab = documento[documento.index('\nAB')+3:documento.index('\nRF')].split()
                except:
                    ab = documento[documento.index('\nAB')+3:documento.index('\nCT')].split()                  
            except:
                try:
                    ab = documento[documento.index('\nEX')+3:documento.index('\nRF')].split()
                except:
                    try:
                        ab = documento[documento.index('\nEX')+3:].split()
                    except:
                        ab = documento[documento.index('\nAB')+3:].split()

        
            for w in title + mj + mn + ab + autor:
                #print(w.strip(string.punctuation).lower())
                w = w.strip(string.punctuation).lower()
                if not w.isdigit():
                    if w in voc:
                        voc[w] += 1
                    else:
                        voc[w] = 1
            for w in voc:
                if w in listaInv:
                    listaInv[w].append([len(docs), voc[w]])
                else:
                    listaInv[w] = [[len(docs),voc[w]]]
            docs.append(voc)

# gera o arquivo invertido

In [90]:
arquivoInv = open('arqInv.dat','w')
for i in range(0,len(docs)):
    linha = str(i)
    for w in docs[i]:
        linha += ';' + w + ' ' + str(docs[i][w])
    arquivoInv.writelines(linha + '\n')
arquivoInv.close()

# calcula o idf de todas as palavras

In [91]:
idfs = {}
for w in listaInv:
    idfs[w] = math.log(len(docs)/len(listaInv[w]))

# pergunta

In [92]:

#pergunta = 'What are the effects of calcium on the physical properties of mucus from CF patients?'.strip(string.punctuation).lower().split()
pergunta = 'Are there abnormalities of taste in CF patients?'.strip(string.punctuation).lower().split()
#pergunta = 'What is the incidence of and treatment for hypertrophic osteoarthropathy in CF patients?'.strip(string.punctuation).lower().split()
#print(pergunta)
queries = []
arqPerguntas = open('cfc/cfquery','r')
perguntas = arqPerguntas.read()
perguntas = perguntas.split('QN')

for i in range(1, len(perguntas)):
    queries.append(perguntas[i][perguntas[i].index('QU')+3:perguntas[i].index('NR')].strip(string.punctuation).lower().split())

sorted(getRelevantes(100), key=itemgetter(1))



[['1033', 1],
 ['59', 2],
 ['803', 2],
 ['833', 2],
 ['1097', 2],
 ['1232', 2],
 ['370', 5],
 ['1000', 6],
 ['1017', 7],
 ['183', 8],
 ['579', 8]]

# vetor da consulta

In [93]:
q = []
palavrasLidas = []
for w in pergunta:
    if w not in palavrasLidas:
        q.append(idfs[w] * pergunta.count(w))
    palavrasLidas.append(w)
pergunta = list(OrderedDict.fromkeys(pergunta))

print(q)
print(pergunta)

[0.9358512577286487, 1.7104138297741025, 2.8179947884249725, 0.013815741897601822, 5.7357655205092515, 0.035321947118565324, 1.216698033574572, 0.512710638461762]
['are', 'there', 'abnormalities', 'of', 'taste', 'in', 'cf', 'patients']


# vetor documentos

In [94]:
vetDocs = []

for doc in docs:
    v = []
    for w in pergunta:
        if w in doc:
            v.append(doc[w] * idfs[w])
        else:
            v.append(0)
    vetDocs.append(v)

# similaridade

In [95]:
sim = []
for i in range(0,len(docs)):
    try:
        sim.append(multVet(vetDocs[i],q)/(norma(vetDocs[i])*norma(q)))
    except:
        sim.append(0)


# TOP 5

In [96]:
top_5_idx = np.argsort(sim)[-5:]
top_5_idx

array([483, 991, 448, 267, 323])

# TESTEs

In [18]:
listaInv['what']

[[55, 1],
 [105, 1],
 [133, 1],
 [297, 1],
 [376, 1],
 [464, 1],
 [489, 1],
 [504, 1],
 [725, 1],
 [734, 1],
 [766, 1],
 [910, 1],
 [925, 1],
 [930, 1],
 [942, 1],
 [974, 1],
 [1030, 2],
 [1110, 1],
 [1147, 1],
 [1217, 1]]

In [89]:
len(voc)


a = list(OrderedDict.fromkeys(['caio','octavio','caio','aelmao']))

In [59]:
len(docs)

1239

In [97]:
print(sim[0])

0.07503193086705001
